Подключениек гуглу-диску:

In [ ]:
from google.colab import drive



drive.mount('noises/')
root_path = 'noises/My Drive/'  #change dir to your project folder

Mounted at noises/


Текущая директория

In [ ]:
!rm -r noises

In [ ]:
!unzip noises1.zip

Archive:  noises1.zip
  inflating: noises/ofis3.wav        
  inflating: noises/morning_village2.wav  
  inflating: noises/factory1.wav     
  inflating: noises/metro5.wav       
  inflating: noises/morning_village4.wav  
  inflating: noises/ofis2.wav        
  inflating: noises/wave_beach1.wav  
  inflating: noises/factory3.wav     
  inflating: noises/train2.wav       
  inflating: noises/kassa4.wav       
  inflating: noises/coughing1.wav    
  inflating: noises/tram5.wav        
  inflating: noises/metro1.wav       
  inflating: noises/morning_village3.wav  
  inflating: noises/fen1.wav         
  inflating: noises/Fart2.wav        
  inflating: noises/coughing2.wav    
  inflating: noises/factory2.wav     
  inflating: noises/fen5.wav         
  inflating: noises/xerox3.wav       
  inflating: noises/Fart4.wav        
  inflating: noises/factory4.wav     
  inflating: noises/coughing5.wav    
  inflating: noises/Fart3.wav        
  inflating: noises/wave_beach4.wav  
  inflating: 

Перемещаемся в папку с нашими файлами в гугл-диске

In [ ]:

%cd noises/MyDrive/

/content/noises/MyDrive


In [ ]:
from IPython.display import Audio
Audio('/content/noises/MyDrive/data/lisa/data/timit/raw/TIMIT/TEST/DR1/FAKS0/SA1.WAV')

In [ ]:
import pandas as pd

Функция для определния RMS – это параметр, показывающий среднюю громкость звучания трека или какой-либо его части. С математической точки зрения RMS (Root Mean Square) – это среднеквадратическое значение громкостей всех семплов дорожки. Подробнее здесь: http://danalex.ru/rms/

In [ ]:
def get_rms(signal):
  rms = np.sqrt(np.mean(np.square(signal)))
  return np.double(rms)

In [ ]:
!pip install soundfile
import soundfile as sf

Функция для определения snr - Отношение сигнал / шум ( SNR или S / N ) - это показатель, используемый в науке и технике, который сравнивает уровень полезного сигнала с уровнем фонового шума . SNR определяется как отношение мощности сигнала к мощности шума , часто выражаемое в децибелах . Соотношение сигнал шум - https://ru.qaz.wiki/wiki/Signal-to-noise_ratio

In [ ]:
import math
def get_snr(noise_signal, clean_signal):
  CL = np.mean(np.square(clean_signal))
  SN = np.mean(np.square(noise_signal))
  N = np.mean(np.square(clean_signal-noise_signal))
  return round((10*math.log10(SN/N)),2)

Читаем чистый аудиосигнал с речью и выводим такие показатели, как имя файла, массив амплитуд сигнала (размер массива равный частота дискретизации на количество сек), путь файла, длительность в сек, средняя мощнось сигнала, класс (если чистые сигналы были нарезаны с разных источников), пол говорящего

In [ ]:
def read_clean(audio):
  #print(audio)
  arr = audio.split(',')
  x = librosa.load(os.path.join(arr[-1],os.path.basename(arr[0])), sr=16000)[0]
  return (
      {
          'name': os.path.basename(arr[0]),
          'signal': x,
          'path' : os.path.join(arr[-1],os.path.basename(arr[0])),
          'duration' : arr[2],
          'power_db' : arr[3],
          'class': arr[1],
          'genre': arr[4]
      }
  )

In [ ]:
import numpy as np
import random
import os

In [ ]:
pip install apache_beam

     |████████████████████████████████| 9.0MB 5.1MB/s 
     |████████████████████████████████| 153kB 49.4MB/s 
     |████████████████████████████████| 2.2MB 36.0MB/s 
     |████████████████████████████████| 829kB 38.1MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 17.7MB 236kB/s 
     |████████████████████████████████| 112kB 34.8MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78533 sha256=37c061d6cc6c30314524e91465818020ae1f31f02ee403fae0cadc18f2351027
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=35a0469511701af57e6a682ebe1f084fe70c0c52570c9f6ea7573cf39fdd9051
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for avro-python3: filenam

In [ ]:
import apache_beam as beam
import pandas as pd


In [ ]:
import shutil

Разделяем шумы на несколько файлов csv, чтобы при зашумлении в итоге у нас получилось примерно равное количество зашумленных файлов по классам. 
```
import os
k=0
for i in range(28):
  df2=df.loc[k:k+36]
  df2.to_csv(os.path.join('noises_32_02_csvs',"file{}.csv".format(k)),encoding='utf-8', sep=',', index=False)
  k=k+35
```
Расчет range идет как количество всего аудиофайлов шумов (у нас их 965) делим на количество шумов,которое набираем в один csv файл (брала 35 штук)
'noises_32_02_csvs' - это название папки где будут файлы.

Теперь нам надо сформировать список этих файлов и запустить конвейер. Список формируется командой ниже:

In [ ]:
import os
files = os.listdir('noises_32_02_csvs')

In [ ]:
files = ['file0.csv',
 'file36.csv',
 'file72.csv',
 'file108.csv',
 'file144.csv',
 'file216.csv',
 'file180.csv',
 'file540.csv',
 'file576.csv',
 'file324.csv',
 'file468.csv',
 'file432.csv',
 'file288.csv',
 'file504.csv',
 'file252.csv',
 'file396.csv',
 'file360.csv',
 'file648.csv',
 'file684.csv',
 'file720.csv',
 'file756.csv',
 'file792.csv',
 'file828.csv',
 'file864.csv',
 'file900.csv',
 'file612.csv',
 'file936.csv']

Класс NoiseProvider с конструктором, который читает файл csv с шумами и формирует список всех прочтенных шумов и с помощю метода provide читает случайно выбранный аудио файл с шумом из сформированного списка.
Тут же вызывается _provide - он формирует данные о файле с шумом: массив амплитуд, имя файла, путь, длительность, мощность, тип шума. 
После получения характеристик, мы смотрим стоит ли обрезать шум или обнулить его слева и справа, чтобы размер массива его амплитуд совпадало с размером массива амплитуд аудио с чистой речью.

In [ ]:
import io
class NoiseProvider:
    def __init__(self, name):
      with open("noises_32_02_csvs/{}".format(name)) as f:
            next(f) 
            self.noises=list(map(lambda l: l.strip(), f.readlines()))


    def _provide(self, audio):
          arr = audio.split(',')
          #print(arr)
          x = librosa.load(arr[1], sr=16000)[0]
          arr1 = os.path.basename(arr[1]).split('.')
          str_ = arr1[0]
          
          aud = {'signal': x,
              'name': str_,
              'path' : arr[1],
              'duration' : arr[4],
              'power_db' : round(10*math.log10(np.mean(np.square(x)))*(-1),2),
              'type_' : arr[0]}
          return aud
    
    def provide(self, trgt_shape):
        noise_signal = self._provide(np.random.choice(self.noises))
        noise_signal_shape = noise_signal['signal'].shape[0]
        if noise_signal_shape < trgt_shape:
            lpad = (trgt_shape - noise_signal_shape)//2
            rpad = trgt_shape - noise_signal_shape - lpad
            noise_signal['signal'] = np.pad(noise_signal['signal'], (lpad, rpad))
        elif noise_signal_shape > trgt_shape:
            noise_signal['signal'] = noise_signal['signal'][:trgt_shape]
        return noise_signal

Метод который зашумляет аудио файл. Мы видим, что выбирвется рандомный snr (или можно не рандомный, а один какой-о выбрать), инициализируем объект класса NoiseProvider. В name указываем название файла списка с шумами. 

noise=noise_provider.provide(cl['signal'].shape[0]) - вызываем метод и отправляем размера массива чистого прочтенного нами аудиофайла

Формируем название файла. В 'noises_files_13_02/all' указывем папку для сохранения нового зашумленного аудифайла. Имя состоит из названия шума_имя чистого аудио сигнала.
slit - функция которая разбиваем строку на массив по заданному символу (по точке в данном случае). Нужно чтобы выделитьимя без его формата.

Дальше иде процедура зашумления файла по заданному snr (разбирали в декабре)

Формируем описание получившегося зашумленного аудиосигнала: название, путьчистого аудиосгнала, snr, путь шума, ощность сигнала, длительность, ип шума, пол говорящего, класс чистого сигнала, мощность зашумленного сигнала, мощность чистого сигнала.

Добавляем описание в список, который в дальнейшем будет сохранен в виде csv файла. Возвращаем только амплитуду иназвание файла.

In [ ]:
import random
def noise_snr(cl, name):
  #print(cl)
  snr = random.randrange(-5,6,5)
  noise_provider = NoiseProvider(name)
  noise=noise_provider.provide(cl['signal'].shape[0])
  new_file = os.path.join('noises_files_13_02/all',"{}_{}".format(os.path.basename(noise['path']).split('.')[0],cl['name']))
  rms_c = get_rms(cl['signal'])
  rms_n = get_rms(noise['signal'])
  noisy_rms = rms_c/(10**(float(snr)/20))
  adjusted_noise = noise['signal'] * (noisy_rms/rms_n)
  signal = cl['signal']+adjusted_noise
  aud = [new_file, 
                       cl['path'], 
                       get_snr(cl['signal'], signal),
                       noise['path'],
                       round(10*math.log10(np.mean(np.square(signal)))*(-1),2), 
                       len(signal)/16000, 
                       noise['type_'],cl['genre'] ,
                       #cl['genre'], 
                       cl['class'], 
                       noise['power_db'], 
                       cl['power_db']
         ]
  manifest_snr1.append(aud)

  return ({'path' : new_file, 
           'signal' : signal})

Записывем аудиосигнал

In [ ]:
def create_wav(signal):
  sf.write(signal['path'], signal['signal'], 16000)


Сам конвеер. С помощью for мы перебираем список файлов с содержанием шумов, и запускаем конвеер.
Сначала читаем мнифест с чистыми аудиосигналами. Пропускаем заголовочную часть(шапку таблицы). Читаем чистый аудиосигнал и выводим основную информацию о нем. Зашумляем аудиофайл, при этом name=i указываем название списка шумов. Записываем зашумленный файл.

Сохраняем ранее созданный список с описанием зашумленных файл в файл csv. Колонки задаем в columns_snr.

Здесь указываем папку сохранения noises_files_13_02_csv/manifest_beam_{} и название файла. В {} указываем имя списка шумов.

In [ ]:
def unarhiv(str, zip):
  if(zip == 1):
    import zipfile
    z = zipfile.ZipFile(str, 'r')
    z.extractall() 
  else:
    !tar -xvzf str

unarhiv("noises.zip", 1)

In [ ]:
import os
import numpy as np
import librosa
for i in files:
  manifest_snr1 = []
  print(i)
  with beam.Pipeline() as p:
    (p | beam.io.ReadFromText('manifest_clean_signal_1_021.csv', skip_header_lines=1)
       | beam.Map(read_clean)
       | beam.Map(noise_snr, name=i)
       | beam.ParDo(create_wav))
  columns_snr = ['path', 'clean_path','snr', 'noise_path', 'power_db','duration', 'type_', 'genre', 'class_clean', 'power_db_noise', 'power_db_clean']
  df_csv = pd.DataFrame(data = manifest_snr1, columns =columns_snr)
  df_csv.to_csv("noises_files_13_02_csv/manifest_beam_{}".format(i), encoding='utf-8', sep=',', index=False)

Что делать со списком? Объединяем их в один список и формируем общий датасет:
<li>Считываем все файлы .csv в папке</li>
<li>формируем один общий датасет</li>
<li>записываем и сохраняем в csv</li>

```
import glob
import pandas as pd
csvs = glob.glob('noises_files_13_02_csv/*.csv')
dfs = (pd.read_csv(f) for f in csvs)
df = pd.concat(dfs, ignore_index=True)
df.to_csv('all.csv', encoding='utf-8', sep=',', index=False)
```